In [1]:
import time
import requests
import pymongo

In [108]:
import pandas as pd
import numpy as np

In [205]:
import pickle

In [2]:
class EventAPIClient:
    """Realtime Events API Client"""

    def __init__(self, first_sequence_number=0,
                 api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/',
                 api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC',
                 db=None,
                 interval=30):
        """Initialize the API client."""
        self.next_sequence_number = first_sequence_number
        self.api_url = api_url
        self.api_key = api_key
        self.db = db
        self.interval = 30

    def save_to_database(self, row):
        """Save a data row to the database."""
        print("Received data:\n" + repr(row) + "\n")  # replace this with your code

    def get_data(self):
        """Fetch data from the API."""
        payload = {'api_key': self.api_key,
                   'sequence_number': self.next_sequence_number}
        response = requests.post(self.api_url, json=payload)
        data = response.json()
        self.next_sequence_number = data['_next_sequence_number']
        return data['data']

    def collect(self, interval=30):
        """Check for new data from the API periodically."""
        while True:
            print("Requesting data...")
            data = self.get_data()
            if data:
                print("Saving...")
                for row in data:
                    self.save_to_database(row)
            else:
                print("No new data received.")
            print(f"Waiting {interval} seconds...")
            time.sleep(interval)


def main():
    """Collect events every 30 seconds."""
    client = EventAPIClient()
    client.collect()

In [85]:
next_sequence_number=0
api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'
stor={}
payload = {'api_key': api_key,
                       'sequence_number': next_sequence_number}
response = requests.post(api_url, json=payload)
data = response.json()
stor[next_sequence_number] = data
next_sequence_number = data['_next_sequence_number']

In [258]:
def get_data():
    api_url='https://hxobin8em5.execute-api.us-west-2.amazonaws.com/api/'
    api_key='vYm9mTUuspeyAWH1v-acfoTlck-tCxwTw9YfCynC'
    next_sequence_number=0
    payload = {'api_key': api_key,
                   'sequence_number': 0}
    response = requests.post(api_url, json=payload)
    data = response.json()
    stor=pd.DataFrame.from_dict(data['data'])
    arr=[0,]
    for i in range(1500,1800):
        
        """Fetch data from the API."""
        payload = {'api_key': api_key,
                   'sequence_number': i}
        response = requests.post(api_url, json=payload)
        temp_data = response.json()
        temp_df=pd.DataFrame.from_dict(temp_data['data'])
        stor=pd.concat([stor,temp_df])
        next_sequence_number = data['_next_sequence_number']
        arr.append(next_sequence_number)
    return stor

In [259]:
a = get_data()

In [282]:
a=a.loc[a.astype(str).drop_duplicates().index]

In [283]:
df=a

In [284]:
if 'acct_type' not in df.columns:
                df['acct_type'] = df.iloc[:, 0].apply(lambda x: 'unknown')
df['is_fraud'] = df['acct_type'].apply(lambda x: 1 if 'fraud' else ('unknown' if 'unknown' else 0))     

In [285]:
required_columns=['body_length', 'channels', 'delivery_method', 'event_published',
       'fb_published', 'has_analytics', 'has_logo',
       'name_length', 'num_order', 'num_payouts', 'org_facebook',
       'org_twitter', 'sale_duration', 'sale_duration2', 'show_map',
       'user_age', 'user_created', 'user_type']

In [286]:
len(required_columns)

18

In [287]:
for col in required_columns:
        if col not in df.columns:
                df[col] = df.iloc[:, 0].apply(lambda x: 0)

In [288]:
X=df[required_columns]

In [289]:
X.shape

(21, 18)

In [290]:
df.acct_type.nunique()

1

In [291]:
model_rf = pickle.load(open('rf.pkl','rb'))

In [292]:
y_rf = model_rf.predict(X)

In [293]:
y_rf

array([0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0])

In [294]:
sum(y_rf)

13

In [295]:
y_rf_prob = model_rf.predict_proba(X)

In [296]:
y_rf_prob

array([[0.51965066, 0.48034934],
       [0.17030568, 0.82969432],
       [0.36681223, 0.63318777],
       [0.2154294 , 0.7845706 ],
       [0.47598253, 0.52401747],
       [0.56186317, 0.43813683],
       [0.18195051, 0.81804949],
       [0.35080058, 0.64919942],
       [0.45123726, 0.54876274],
       [0.54439592, 0.45560408],
       [0.51965066, 0.48034934],
       [0.36681223, 0.63318777],
       [0.2154294 , 0.7845706 ],
       [0.47598253, 0.52401747],
       [0.56186317, 0.43813683],
       [0.18195051, 0.81804949],
       [0.35080058, 0.64919942],
       [0.45123726, 0.54876274],
       [0.54439592, 0.45560408],
       [0.51965066, 0.48034934],
       [0.55604076, 0.44395924]])